In [ ]:
# Importing required libraries and functions

import os 
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import sum, col, month, year, dayofweek, dayofmonth, hour, unix_timestamp, median, mode, lit, when

In [ ]:
spark = SparkSession.builder.appName("ETL").master("local[*]").getOrCreate()  

# Setting AWS-Access Credentials
spark.conf.set("fs.s3a.access.key", str(os.environ['AWS_ACCESS_KEY']))
spark.conf.set("fs.s3a.secret.key", str(os.environ['AWS_SECRET_ACCESS_KEY']))
spark.conf.set("fs.s3a.endpoint", "s3.amazonaws.com")

# Read the green taxi parquet files into a DataFrame
GREEN_TAXI_PATH = "s3a://data228/raw-taxi-data/green-taxi/"
green_taxi_df = spark.read.parquet(GREEN_TAXI_PATH)
green_taxi_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [ ]:
gc = green_taxi_df.count()
print(f"Number of records in Green Taxi data: {gc}")

Number of records in Green Taxi data: 1909157


In [ ]:
# Read the yellow taxi parquet files into a DataFrame 

YELLOW_TAXI_PATH = "s3a://data228/raw-taxi-data/yellow-taxi/"
yellow_taxi_df = spark.read.parquet(YELLOW_TAXI_PATH)
yellow_taxi_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [ ]:
yc = yellow_taxi_df.count()
print(f"Number of records in Yellow Taxi data: {yc}")

Number of records in Yellow Taxi data: 70560406


In [ ]:
# Making column names consistent on both datasets

green_taxi_df = green_taxi_df \
    .withColumnRenamed("lpep_pickup_datetime", "tpep_pickup_datetime") \
    .withColumnRenamed( "lpep_dropoff_datetime", "tpep_dropoff_datetime")

green_taxi_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [ ]:
# Removing tip amount from total_amount

yellow_taxi_df = yellow_taxi_df.withColumn('total_amount', lit(col('total_amount') - col('tip_amount')))
green_taxi_df = green_taxi_df.withColumn('total_amount', lit(col('total_amount') - col('tip_amount')))

In [ ]:
# Selecting only required columns (feature engineering)

yellow_df = yellow_taxi_df.select(col('tpep_pickup_datetime'), col('tpep_dropoff_datetime'), col('passenger_count'), col('trip_distance'), col('PULocationID'), col('DOLocationID'), col('payment_type'), col('total_amount'))
green_df = green_taxi_df.select(col('tpep_pickup_datetime'), col('tpep_dropoff_datetime'), col('passenger_count'), col('trip_distance'), col('PULocationID'), col('DOLocationID'), col('payment_type'), col('total_amount'))

In [ ]:
# Merging green and yellow taxi data
merged_df = yellow_df.unionByName(green_df)

# Find the number of records with null values
null_counts = merged_df.select([sum(col(column).isNull().cast("int")).alias(column) for column in merged_df.columns])
null_counts.show()

+--------------------+---------------------+---------------+-------------+------------+------------+------------+------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|payment_type|total_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+------------+------------+
|                   0|                    0|        3349772|            0|           0|           0|      502774|           0|
+--------------------+---------------------+---------------+-------------+------------+------------+------------+------------+



In [ ]:
merged_df.select(median('passenger_count'), mode('payment_type')).show()

+-----------------------+------------------+
|median(passenger_count)|mode(payment_type)|
+-----------------------+------------------+
|                    1.0|               1.0|
+-----------------------+------------------+



In [ ]:
# Since the number of records is relatively minimal, we'll impute these missing values with median/mode values

df = merged_df.withColumn('passenger_count', when(col('passenger_count').isNull(), lit(1)).otherwise(col('passenger_count'))) \
    .withColumn('payment_type', when(col('payment_type').isNull(), lit(1)).otherwise(col('payment_type')))

In [ ]:
# Creating trip_duration field in seconds

df = df \
    .withColumn("trip_duration", (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))) \
    .withColumn('year', year("tpep_pickup_datetime")) \
    .withColumn('month', month("tpep_pickup_datetime")) \
    .withColumn('weekend', when((dayofweek("tpep_pickup_datetime") == 1) | (dayofweek("tpep_pickup_datetime") == 7), lit(1)).otherwise(lit(0))) \
    .withColumn('date', dayofmonth("tpep_pickup_datetime")) \
    .withColumn('hour', hour("tpep_pickup_datetime"))


In [ ]:
# Summary Statistics of few important columns

df.select('passenger_count', 'trip_distance', 'total_amount', 'trip_duration').summary("count", "mean", "min", "25%", "75%", "99%", "99.9%", "max").show()

+-------+-----------------+-----------------+------------------+------------------+
|summary|  passenger_count|    trip_distance|      total_amount|     trip_duration|
+-------+-----------------+-----------------+------------------+------------------+
|  count|         72469563|         72469563|          72469563|          72469563|
|   mean|1.390948486884073|9.368861344038088|15.837589910634756|2216.1962188456964|
|    min|              0.0|              0.0|    -1.333914107E8|          -8105078|
|    25%|              1.0|              1.1|              10.3|               427|
|    75%|              1.0|             3.48|             19.63|              1143|
|    99%|              6.0|            20.02| 67.60000000000001|              3889|
|  99.9%|              6.0|             29.8|             120.3|             84112|
|    max|            112.0|        389678.46|         818286.74|         619323311|
+-------+-----------------+-----------------+------------------+------------

#### Few Observations from this summary statistics
- We can see outliers in few columns
- Maximum paassenger_count is **112**. Is it a taxi or bus?
- Maximum trip duration is **619M** seconds
- Total amount in negatives? And max amount is over **$800K**? Taxi trip to Moon? 

Next step is to remove these outliers

In [ ]:
# Filtering out outliers

# Filter records with passenger_count between 0 and 7
df = df.filter(df['passenger_count'] <= 7) \
    .filter(df['passenger_count'] > 0)

# Filter records with trip_distance more than 70 miles
df = df.filter(df['trip_distance'] < 70) \
    .filter(df['trip_distance'] > 0.25)

# Filtering out records with sub zero total_amount and more than $500
df = df.filter(df['total_amount'] > 0) \
    .filter(df['total_amount'] < 500)

# Filtering out records with duration less than 1 minute or more than 5hrs
df = df.filter(df['trip_duration'] > 60) \
    .filter(df['trip_duration'] < 18000)

df = df.filter(df['year'] <= 2023) \
    .filter(df['year'] > 2018)

c = df.count()
print(f"Number of records after removing outliers: {c}")

Number of records after removing outliers: 68891849


In [ ]:
# Group by year and month and perform aggregation

aggregated_data = df.groupBy(year('tpep_pickup_datetime').alias('year'), month('tpep_pickup_datetime').alias('month')) \
    .agg({
        'passenger_count' : 'count',
        'passenger_count' : 'sum',
        'total_amount': 'sum',
        'trip_distance': 'sum'
    })

display(aggregated_data)

year,month,sum(total_amount),sum(passenger_count),sum(trip_distance)
2022,10,6.885363204656516E7,4962217.0,1.3123310930000668E7
2022,2,4.953080734898312E7,4093472.0,9165898.999999082
2022,11,6.056083980765324E7,4407348.0,1.146978568000014E7
2022,3,6.3434998327058904E7,4965689.0,1.2037523720000518E7
2022,5,6.717364115697E7,4905466.0,1.2972053440000756E7
2022,9,6.0996660607714824E7,4318436.0,1.169427119000073E7
2022,4,6.476951470700639E7,5004934.0,1.2404447259999676E7
2022,12,7.109628053988962E7,4699547.0,1.1989602089999825E7
2023,1,83.8,9.0,8.6
2022,7,5.7776307767914385E7,4436571.0,1.1361056989999546E7


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [ ]:
# Reading historical weather data

WEATHER_PATH = "s3://data228/raw-weather-data/weather.csv"
weather_df = spark.read.csv(WEATHER_PATH, header=True, inferSchema=True)
weather_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- temp: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: integer (nullable = true)
 |-- preciptype: string (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- sealevelpressure: double (nullable = true)
 |-- cloudcover: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- uvindex: integer (nullable = true)
 |-- severerisk: integer (nullable = true)
 |-- conditions: string (nullable = true)
 |-- icon: string (nullable = true)
 |-- stations: string (nullable = true)



In [ ]:
weather_df = weather_df.select('datetime', 'temp', 'dew', 'humidity', 'windspeed', 'visibility')
weather_df.show(5)

+-------------------+----+----+--------+---------+----------+
|           datetime|temp| dew|humidity|windspeed|visibility|
+-------------------+----+----+--------+---------+----------+
|2021-01-01 00:00:00|37.1|25.9|   63.58|      7.8|       9.9|
|2021-01-01 01:00:00|37.1|25.9|   63.58|      0.1|       9.9|
|2021-01-01 02:00:00|37.1|25.0|    61.3|      0.3|       9.9|
|2021-01-01 03:00:00|37.0|25.0|   61.33|      8.9|       9.9|
|2021-01-01 04:00:00|37.0|24.9|   61.24|     10.0|       9.9|
+-------------------+----+----+--------+---------+----------+
only showing top 5 rows



In [ ]:
# Merging weather data and taxi data

df = df.withColumn('hour', hour('tpep_pickup_datetime'))
weather_df = weather_df \
    .withColumn('year', year('datetime')) \
    .withColumn('month', month('datetime')) \
    .withColumn('date', dayofmonth('datetime')) \
    .withColumn('hour', hour('datetime')) \

final_df = df.join(weather_df, on=['year', 'month', 'date', 'hour'], how='inner')

In [ ]:
final_df = final_df.drop('datetime')
final_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- trip_duration: long (nullable = true)
 |-- weekend: integer (nullable = false)
 |-- temp: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- visibility: double (nullable = true)



In [ ]:
# Saving the merged final dataset to new S3 directory

PATH = "s3://data228/final-data/"
final_df.coalesce(1).write.parquet(PATH, mode='overwrite')